# HW 5 - Building a normalized RDB
 
The goal of this homework is to take a semi-structured non-normalized CSV file and turn it into a set of normalized tables that you then push to your postgres database on AWS.  
 
The original dataset contains 100k district court decisions, although I've downsampled it to only 1000 rows to make the uploads faster.  Each row contains info about a judge, their demographics, party affiliation, etc.  Rows also contain information about the case they were deciding on.  Was it a criminal or civil case?  What year was it?  Was the direction of the decision liberal or conservative?  
 
While the current denormalized format is fine for analysis, it's not fine for a database as it violates many normalization rules. Your goal is to normalize it by designing a simple schema, then wrangling it into the proper dataframes, then pushing it all to AWS.  
 
For the first part of this assignment you should wind up with three tables.  One with case information, one with judge information, one that has casetype information, and the last with case category information.  Each table should be reduced so that there are not then repeating rows, and primary keys should be assigned within each.  These tables should be called 'cases', 'judges', and 'casetype'.
 
For the last part you should make a rollup table that calculates the percent of liberal decisions for each party level and each case category.  This will allow for one to get a quick look at how the political party affiliation of judges impacts the direction of a decision for different case categories (e.g. criminal, civil, labor). 

## Bring in data, explore, make schema

Start by bringing in your data to `cases`. Call a `.head()` on it to see what columns are there and what they contain.

In [1]:
import pandas as pd
import psycopg2
import numpy as np
cases = pd.read_csv('https://ista322data.s3-us-west-1.amazonaws.com/cases_denorm_small.csv')

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
# head of cases
cases.head()

,judge_name,party_name,gender_name,race_name,case_id,case_year,casetype_id,casetype_name,category_id,category_name,libcon_id,libcon_name
0,"Thompson, Myron H.",Democrat,male,African-American/black,28321332,2011,3,criminal court motions,1,Criminal Justice Cases,0,Conservative
1,"Shoob, Marvin A.",Democrat,male,white/caucasian,18110669,1993,14,free of religion,2,Civil Liberties/Rights Cases,1,Liberal
2,"Bua, Nicholas J.",Democrat,male,white/caucasian,15660871,1983,2,habeas corpus-state,1,Criminal Justice Cases,0,Conservative
3,"Kovachevich, Elizabeth",Republican,female,white/caucasian,17770934,1991,2,habeas corpus-state,1,Criminal Justice Cases,0,Conservative
4,"Gilliam, Earl B.",Independent/Other/Unknown,male,white/caucasian,26621195,2009,6,alien petitions,2,Civil Liberties/Rights Cases,0,Conservative


### Make schema
 
OK, given that head, you need to make three related tables that will make up a normalized database.  Those tables are 'cases', 'judges', and 'casetype'.  If it's not clear what info should go into each, explore the data more, ask on slack, etc.  
 
Remember, you might not have keys, will need to reduce the rows, select certain columns, etc.  There isn't a defined path here.  

Here's a textbox to define what should be in your schema... feel free to add to this. 


## Make cases table - 3 points

Start by making a table that contains just each case's info.  I would call this table that you're going to upload `cases_df` so you don't overwrite your raw data.  

This table should have six columns and 1000 rows.  

Note, one of these columns should be a judge_id that links to the judges table. You'll need to make this foreign key.  

Also, you can leave 'category_name' in this table as well as its id.  Normally you'd split that off into it's own table as well, but you're already doing that for casetype which is enough for now.  

In [3]:
# Make judge_id in cases
cases['judge_id'] = pd.factorize(cases['judge_name'])[0].astype(str)


In [15]:
# select necessary columns to make cases_df


cases_df = cases.copy()
#cases_df = cases_df.set_index('case_id')
cases_df = cases_df.drop(columns=['party_name', 'gender_name', 'race_name', 'libcon_id', 'libcon_name', 'judge_name', 'casetype_name'])

In [20]:
# What's the shape of cases_df?
cases_df.shape

(1000, 6)

### Make cases table in your database

Go bring over your connection function from the last workbook. **This must be in here for me to grade the homework** If I can't access your database you'll get a zero.  It's a good idea to bring our other exploratory functions as well.  

Once you do that you'll need to do the following

* Connect, make a table called 'cases' with the correct column names and data types.  Be sure to execute and commit the table.  
* Make tuples of your data
* Write a SQL string that allows you to insert each tuple of data into the correct columns
* Execute the string many times to fill out 'cases'
* Commit changes and check the table.  

I'm not going to leave a full roadmap beyond this.  Feel free to add cells as needed to do the above.  

In [6]:
def get_conn_cur(): # define function name and arguments (there aren't any)
  # Make a connection
  conn = psycopg2.connect(
    host="test-hw-db.codpumsddvml.us-east-1.rds.amazonaws.com",
    database="hw3_db_newton",
    user="infamous",
    password="Viperpilot36",
    port='5432')
  
  cur = conn.cursor()   # Make a cursor after

  return(conn, cur)   # Return both the connection and the cursor

# run_query function
def run_query(query_string):

  conn, cur = get_conn_cur() # get connection and cursor

  cur.execute(query_string) # executing string as before

  my_data = cur.fetchall() # fetch query data as before

  # here we're extracting the 0th element for each item in cur.description
  colnames = [desc[0] for desc in cur.description]

  cur.close() # close
  conn.close() # close

  return(colnames, my_data) # return column names AND data

# Column name function for checking out what's in a table
def get_column_names(table_name): # arguement of table_name
  conn, cur = get_conn_cur() # get connection and cursor

  # Now select column names while inserting the table name into the WERE
  column_name_query =  """SELECT column_name FROM information_schema.columns
       WHERE table_name = '%s' """ %table_name

  cur.execute(column_name_query) # exectue
  my_data = cur.fetchall() # store

  cur.close() # close
  conn.close() # close

  return(my_data) # return

# Check table_names
def get_table_names():
  conn, cur = get_conn_cur() # get connection and cursor

  # query to get table names
  table_name_query = """SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public' """

  cur.execute(table_name_query) # execute
  my_data = cur.fetchall() # fetch results

  cur.close() #close cursor
  conn.close() # close connection

  return(my_data) # return your fetched results
  
# make sql_head function
def sql_head(table_name):
  conn, cur = get_conn_cur() # get connection and cursor

  # Now select column names while inserting the table name into the WERE
  head_query =  """SELECT * FROM %s LIMIT 5; """ %table_name

  cur.execute(head_query) # exectue
  colnames = [desc[0] for desc in cur.description] # get column names
  my_data = cur.fetchall() # store first five rows

  cur.close() # close
  conn.close() # close

  df = pd.DataFrame(data = my_data, columns = colnames) # make into df

  return(df) # return

conn, cur = get_conn_cur()
conn

<connection object at 0x7fe314fb17d0; dsn: 'user=infamous password=xxx dbname=hw3_db_newton host=test-hw-db.codpumsddvml.us-east-1.rds.amazonaws.com port=5432', closed: 0>

In [23]:
# make table
conn, cur = get_conn_cur()
tq = """CREATE TABLE cases (
          case_id INTEGER PRIMARY KEY,
          case_year INTEGER NOT NULL,
          casetype_id INTEGER NOT NULL,
          category_id INTEGER NOT NULL,
          category_name VARCHAR(255) NOT NULL,
          judge_id INTEGER NOT NULL
          );"""
cur.execute(tq)
conn.commit()

In [25]:
get_column_names(table_name='cases')
data_tups = [tuple(x) for x in cases_df.to_numpy()]
data_tups[2]

(15660871, 1983, 2, 1, 'Criminal Justice Cases', '2')

In [38]:
# push a row
iq = """INSERT INTO cases (case_id, case_year, casetype_id, category_id, category_name, judge_id) VALUES(%s, %s, %s, %s, %s, %s);"""
conn, cur = get_conn_cur()
cur.executemany(iq, data_tups)
conn.commit()


In [40]:
# Use sql_head to check cases
sql_head(table_name='cases')

,case_id,case_year,casetype_id,category_id,category_name,judge_id
0,28321332,2011,3,1,Criminal Justice Cases,0
1,18110669,1993,14,2,Civil Liberties/Rights Cases,1
2,15660871,1983,2,1,Criminal Justice Cases,2
3,17770934,1991,2,1,Criminal Justice Cases,3
4,26621195,2009,6,2,Civil Liberties/Rights Cases,4


In [ ]:
conn.close()

## Make judges - 2 points

Now make your  judges table from the original `cases` dataframe (not the SQL table you just made).  

Judges should have five columns, including the `judge_id` column you made. There should be 553 rows after you drop duplicates (remember that judges may have had more than one case).  

After you  make the dataset go and push to a SQL table called 'judges'. 

In [71]:
judges = cases.copy()
#cases_df = cases_df.set_index('case_id')
judges = judges.drop(columns=['case_id',	'case_year', 'casetype_id',	'casetype_name', 'category_id', 'category_name'])
judges = judges[['judge_id', 'judge_name', 'party_name', 'gender_name', 'race_name', 'libcon_id', 'libcon_name']]
judges

,judge_id,judge_name,party_name,gender_name,race_name,libcon_id,libcon_name
0,0,"Thompson, Myron H.",Democrat,male,African-American/black,0,Conservative
1,1,"Shoob, Marvin A.",Democrat,male,white/caucasian,1,Liberal
2,2,"Bua, Nicholas J.",Democrat,male,white/caucasian,0,Conservative
3,3,"Kovachevich, Elizabeth",Republican,female,white/caucasian,0,Conservative
4,4,"Gilliam, Earl B.",Independent/Other/Unknown,male,white/caucasian,0,Conservative
...,...,...,...,...,...,...,...
995,550,"Weiner, Charles R.",Democrat,male,white/caucasian,0,Conservative
996,314,"Rovner, Illana D.",Republican,female,white/caucasian,0,Conservative
997,551,"Karas, Kenneth M.",Republican,female,white/caucasian,0,Conservative
998,552,"White, George W.",Democrat,male,African-American/black,0,Conservative


In [77]:
judges = judges.drop_duplicates('judge_id')

In [52]:
# make table
conn, cur = get_conn_cur()
tq = """CREATE TABLE judges (
          judge_id INTEGER PRIMARY KEY,
          judge_name VARCHAR(255) NOT NULL,
          party_name VARCHAR(255) NULL,
          gender_name VARCHAR(255) NOT NULL,
          race_name VARCHAR(255) NOT NULL,
          libcon_id INTEGER NOT NULL,
          libcon_name VARCHAR(255) NOT NULL
          );"""
cur.execute(tq)
conn.commit()

In [79]:
get_column_names(table_name='judges')
data_tups = [tuple(x) for x in judges.to_numpy()]
data_tups[552]

('552',
 'White, George W.',
 'Democrat',
 'male',
 'African-American/black',
 0,
 'Conservative')

In [80]:
# push all rows
iq = """INSERT INTO judges (judge_id, judge_name, party_name, gender_name, race_name, libcon_id, libcon_name) VALUES(%s, %s, %s, %s, %s, %s, %s);"""
conn, cur = get_conn_cur()
cur.executemany(iq, data_tups)
conn.commit()

In [81]:
sql_head(table_name='judges')

,judge_id,judge_name,party_name,gender_name,race_name,libcon_id,libcon_name
0,0,"Thompson, Myron H.",Democrat,male,African-American/black,0,Conservative
1,1,"Shoob, Marvin A.",Democrat,male,white/caucasian,1,Liberal
2,2,"Bua, Nicholas J.",Democrat,male,white/caucasian,0,Conservative
3,3,"Kovachevich, Elizabeth",Republican,female,white/caucasian,0,Conservative
4,4,"Gilliam, Earl B.",Independent/Other/Unknown,male,white/caucasian,0,Conservative


In [82]:
conn.close()

## Make casetype - 2 points

Go make the casetype table.  This should have only two columns that allow you to link the casetype name back to the ID in the 'cases' table. There should be 27 rows as well. 

In [87]:
casetype = cases.copy()
casetype = casetype.drop(columns=['judge_id', 'judge_name', 'party_name', 'gender_name', 'race_name', 'libcon_id', 'libcon_name', 'case_id',	'case_year', 'category_id', 'category_name'])
casetype = casetype[['casetype_id',	'casetype_name']]
casetype = casetype.drop_duplicates()

In [88]:
# make table
conn, cur = get_conn_cur()
tq = """CREATE TABLE casetype (
          casetype_id INTEGER PRIMARY KEY,
          casetype_name VARCHAR(255) NOT NULL
          );"""
cur.execute(tq)
conn.commit()

In [91]:
get_column_names(table_name='casetype')
data_tups = [tuple(x) for x in casetype.to_numpy()]
data_tups[0]

(3, 'criminal court motions')

In [92]:
iq = """INSERT INTO casetype (casetype_id, casetype_name) VALUES(%s, %s);"""
conn, cur = get_conn_cur()
cur.executemany(iq, data_tups)
conn.commit()

In [94]:
sql_head(table_name='casetype')

,casetype_id,casetype_name
0,3,criminal court motions
1,14,free of religion
2,2,habeas corpus-state
3,6,alien petitions
4,19,environmental protection


In [96]:
conn.close()

## A quick test of your tables

Below is a query to get the number of unique judges that have ruled on criminal court motion cases.  You should get a value of 119 as your return if your database is set up correctly!

In [95]:
run_query("""SELECT COUNT(DISTINCT(judges.judge_id)) FROM cases 
                JOIN judges ON cases.judge_id = judges.judge_id 
                WHERE casetype_id = (SELECT casetype_id FROM casetype
                                        WHERE casetype_name = 'criminal court motions'); """)
              

(['count'], [(119,)])

## Make rollup table - 3 points
 
Now let's make that rollup table!  The goal here is to make a summary table easily accessed.  We're going to roll the whole thing up by the judges party and the category, but you could imagine doing this for each judge to track how they make decisions over time which would then be useful for an analytics database.  The one we're making could also be used as a dimension table where we needed overall party averages.  
 
We want to get a percentage of liberal decisions by each grouping level (party_name, category_name). To do this we need first, the number of cases seen at each level, and second, the number of liberal decisions made at each level.  `cases` contains the columns `libcon_id` which is a 0 if the decision was conservative in its ruling, and a 1 if it was liberal in its ruling.  Thus, you can get a percentage of liberal decisions if you divide the sum of that column by the total observations.  Your `agg()` can both get the sum and count.  
 
After you groupby you'll need to reset the index, rename the columns, then make the percentage.  
 
Once you do that you can push to a SQL table called 'rollup'
 
Let's get started

In [114]:
# Make a groupby called cases_rollup.  This should group by party_name and categrory name.  It should aggregate the count and sum of libcon_id
cases_rollup = cases.groupby([cases['party_name'], cases['category_name']]).agg({'libcon_id' : ['count', 'sum']})
cases_rollup

libcon_id     
                                                           count  sum
party_name                category_name                              
Democrat                  Civil Liberties/Rights Cases       218  112
                          Criminal Justice Cases             107   36
                          Labor & Economic Cases             126   69
Independent/Other/Unknown Civil Liberties/Rights Cases        12    5
                          Criminal Justice Cases               8    5
                          Labor & Economic Cases              13    6
Republican                Civil Liberties/Rights Cases       237   73
                          Criminal Justice Cases             125   34
                          Labor & Economic Cases             154   67

In [118]:
# reset your index
cases_rollup = cases_rollup.reset_index()

In [121]:
# rename your columns now.  Keep the first to the same but call the last two 'total_cases' and 'num_lib_decisions'
cases_rollup.columns = ['party_name',	'category_name', 'total_cases', 'num_lib_decisions']
cases_rollup

,party_name,category_name,total_cases,num_lib_decisions
0,Democrat,Civil Liberties/Rights Cases,218,112
1,Democrat,Criminal Justice Cases,107,36
2,Democrat,Labor & Economic Cases,126,69
3,Independent/Other/Unknown,Civil Liberties/Rights Cases,12,5
4,Independent/Other/Unknown,Criminal Justice Cases,8,5
5,Independent/Other/Unknown,Labor & Economic Cases,13,6
6,Republican,Civil Liberties/Rights Cases,237,73
7,Republican,Criminal Justice Cases,125,34
8,Republican,Labor & Economic Cases,154,67


Now make a new column called 'percent_liberal'

This should calucalte the percentage of decisions that were liberal in nature.  Multiple it by 100 so that it's a full percent.  Also use the `round()` function on the whole thing to keep it in whole percentages.

In [124]:
# make your metric called 'percent_liberal' 
cases_rollup['percent_liberal'] = 100 * (cases_rollup['num_lib_decisions'] / cases_rollup['total_cases']).round(2)
cases_rollup

,party_name,category_name,total_cases,num_lib_decisions,percent_liberal
0,Democrat,Civil Liberties/Rights Cases,218,112,51.0
1,Democrat,Criminal Justice Cases,107,36,34.0
2,Democrat,Labor & Economic Cases,126,69,55.0
3,Independent/Other/Unknown,Civil Liberties/Rights Cases,12,5,42.0
4,Independent/Other/Unknown,Criminal Justice Cases,8,5,62.0
5,Independent/Other/Unknown,Labor & Economic Cases,13,6,46.0
6,Republican,Civil Liberties/Rights Cases,237,73,31.0
7,Republican,Criminal Justice Cases,125,34,27.0
8,Republican,Labor & Economic Cases,154,67,44.0


Now go and push the whole thing to a table called 'rollup'

There should be five columns and nine rows. 

In [125]:
conn, cur = get_conn_cur()
tq = """CREATE TABLE rollup (
          party_name VARCHAR(255) NOT NULL,
          category_name VARCHAR(255) NOT NULL,
          total_cases INTEGER NOT NULL,
          num_lib_decisions INTEGER NOT NULL,
          percent_liberal INTEGER NOT NULL
          );"""
cur.execute(tq)
conn.commit()
conn.close()

In [133]:
conn, cur = get_conn_cur()
get_column_names(table_name='rollup')
data_tups = [tuple(x) for x in cases_rollup.to_numpy()]
iq = """INSERT INTO rollup (party_name, category_name, total_cases, num_lib_decisions, percent_liberal) VALUES(%s, %s, %s, %s, %s);"""
cur.executemany(iq, data_tups)
conn.commit()
conn.close()

In [134]:
# Check
sql_head('rollup')

,party_name,category_name,total_cases,num_lib_decisions,percent_liberal
0,Democrat,Civil Liberties/Rights Cases,218,112,51
1,Democrat,Criminal Justice Cases,107,36,34
2,Democrat,Labor & Economic Cases,126,69,55
3,Independent/Other/Unknown,Civil Liberties/Rights Cases,12,5,42
4,Independent/Other/Unknown,Criminal Justice Cases,8,5,62
